# 🏋️ AI Search + Agent Service: Fitness-Fun Example 🤸

Welcome to our **AI Search + AI Agent** tutorial, where we'll:

1. **Create** an Azure AI Search index with some fitness-oriented sample data
2. **Demonstrate** how to connect that index to an Agent via the `AzureAISearchTool`
3. **Show** how to query the Agent for health and fitness info in a fun scenario (with disclaimers!)

## 🏥 Health & Fitness Disclaimer
> **This notebook is for general demonstration and entertainment purposes, NOT a substitute for professional medical advice.**
> Always seek the advice of certified health professionals.

## Prerequisites
1. Complete Agent basics notebook - [1-basics.ipynb](1-basics.ipynb)
2. An **Azure AI Search** resource (formerly "Cognitive Search"), provisioned in your Azure AI Foundry project.

## High-Level Flow
We'll do the following:
1. **Create** an AI Search index programmatically with sample fitness data.
2. **Upload** documents (fitness items) to the index.
3. **Create** an Agent that references our new index using `AzureAISearchTool`.
4. **Run queries** to see how it fetches from the index.
 
 <img src="./seq-diagrams/5-ai-search.png" width="30%"/>


## 1. Create & Populate Azure AI Search Index
We'll create a minimal index called `myfitnessindex`, containing a few example items.
Make sure to set your environment variables for `SEARCH_ENDPOINT` and `SEARCH_API_KEY`. We'll use the `azure.search.documents.indexes` classes to manage the index schema. We'll also upload some sample data.


In [ ]:
# Import required Azure libraries
import os
from azure.core.credentials import AzureKeyCredential  # For authentication
from azure.search.documents.indexes import SearchIndexClient  # For managing search indexes
from azure.search.documents.indexes.models import SearchIndex, SimpleField, SearchFieldDataType, SearchableField  # Index schema components
from azure.search.documents import SearchClient  # For document operations (upload/search)
from azure.identity import DefaultAzureCredential  # For Azure authentication
from azure.ai.projects import AIProjectClient  # To access project resources
from azure.ai.projects.models import ConnectionType  # Enum for connection types

# First, initialize the AI Project client which gives us access to project resources
# This uses DefaultAzureCredential for authentication and the project connection string
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

# Get the Azure AI Search connection details from our project
# This includes endpoint URL and API key needed to access the search service
search_conn = project_client.connections.get_default(
    connection_type=ConnectionType.AZURE_AI_SEARCH, 
    include_credentials=True
)
if not search_conn:
    raise RuntimeError("❌ No default Azure AI Search connection found in your project.")

# Name of our search index - this is where our fitness data will be stored
index_name = "myfitnessindex"

try:
    # Create a SearchIndexClient - this is used for managing the index itself (create/update/delete)
    credential = AzureKeyCredential(search_conn.key)
    index_client = SearchIndexClient(endpoint=search_conn.endpoint_url, credential=credential)
    print("✅ Created SearchIndexClient from project_client connection")
    
    # Create a SearchClient - this is used for document operations (upload/search/delete documents)
    # We'll use this later to add our fitness items to the index
    search_client = SearchClient(
        endpoint=search_conn.endpoint_url,
        index_name=index_name,
        credential=credential
    )
    print("✅ Created SearchClient for document operations")
    
except Exception as e:
    print(f"❌ Error creating search clients: {e}")

**Define the index** schema with a `FitnessItemID` key and a few fields to store product info.


In [ ]:
def create_fitness_index():
    # Define the fields (columns) for our search index
    # Each field has specific attributes that control how it can be used in searches:
    fields = [
        # Primary key field - must be unique for each document
        SimpleField(name="FitnessItemID", type=SearchFieldDataType.String, key=True),
        
        # Name field - SearchableField means we can do full-text search on it
        # filterable=True lets us filter results by name
        SearchableField(name="Name", type=SearchFieldDataType.String, filterable=True),
        
        # Category field - SearchableField for text search
        # filterable=True lets us filter by category
        # facetable=True enables category grouping in results
        SearchableField(name="Category", type=SearchFieldDataType.String, filterable=True, facetable=True),
        
        # Price field - SimpleField for numeric values
        # filterable=True enables price range filters
        # sortable=True lets us sort by price
        # facetable=True enables price range grouping
        SimpleField(name="Price", type=SearchFieldDataType.Double, filterable=True, sortable=True, facetable=True),
        
        # Description field - SearchableField for full-text search on product descriptions
        SearchableField(name="Description", type=SearchFieldDataType.String)
    ]

    # Create an index definition with our fields
    index = SearchIndex(name=index_name, fields=fields)

    # Check if index already exists - if so, delete it to start fresh
    # This is useful during development but be careful in production!
    if index_name in [x.name for x in index_client.list_indexes()]:
        index_client.delete_index(index_name)
        print(f"🗑️ Deleted existing index: {index_name}")

    # Create the new index with our schema
    created = index_client.create_index(index)
    print(f"🎉 Created index: {created.name}")

# Execute the function to create our search index
create_fitness_index()

**Upload some sample documents** to `myfitnessindex`. We'll add a few items for demonstration.


In [ ]:
def upload_fitness_docs():
    # Create a SearchClient to interact with our search index
    # This uses the connection details (endpoint, key) we configured earlier
    search_client = SearchClient(
        endpoint=search_conn.endpoint_url,
        index_name=index_name,
        credential=AzureKeyCredential(search_conn.key)
    )

    # Define sample documents that match our index schema
    # Each document must have:
    # - FitnessItemID (unique identifier)
    # - Name (searchable product name) 
    # - Category (searchable and facetable for filtering/grouping)
    # - Price (numeric field for sorting and filtering)
    # - Description (searchable product details)
    sample_docs = [
        {
            "FitnessItemID": "1",
            "Name": "Adjustable Dumbbell",
            "Category": "Strength", 
            "Price": 59.99,
            "Description": "A compact, adjustable weight for targeted muscle workouts."
        },
        {
            "FitnessItemID": "2",
            "Name": "Yoga Mat",
            "Category": "Flexibility",
            "Price": 25.0,
            "Description": "Non-slip mat designed for yoga, Pilates, and other exercises."
        },
        {
            "FitnessItemID": "3",
            "Name": "Treadmill",
            "Category": "Cardio",
            "Price": 499.0,
            "Description": "A sturdy treadmill with adjustable speed and incline settings."
        },
        {
            "FitnessItemID": "4",
            "Name": "Resistance Bands",
            "Category": "Strength",
            "Price": 15.0,
            "Description": "Set of colorful bands for light to moderate resistance workouts."
        }
    ]

    # Upload all documents to the search index in a single batch operation
    # The search service will index these documents, making them searchable
    # based on the field types we defined in our index schema
    result = search_client.upload_documents(documents=sample_docs)
    print(f"🚀 Upload result: {result}")

# Call the function to upload the documents
upload_fitness_docs()
print("✅ Documents uploaded to search index")


### Verify the documents via a basic query
Let's do a quick search for **Strength** items.


In [ ]:
# Let's verify our index by performing a basic search
# 1. First create a SearchClient using our connection details
#    - endpoint_url: The URL of our search service
#    - index_name: The name of the index we created earlier
#    - key: The admin key to authenticate our requests
search_client = SearchClient(
    endpoint=search_conn.endpoint_url,
    index_name=index_name,
    credential=AzureKeyCredential(search_conn.key)
)

# 2. Perform a simple search query:
#    - search_text="Strength": Look for documents containing "Strength"
#    - filter=None: No additional filtering
#    - top=10: Return up to 10 matching documents
results = search_client.search(search_text="Strength", filter=None, top=10)

# 3. Print each matching document
print("🔍 Search results for 'Strength':")
print("-" * 50)
found_items = False
for doc in results:
    found_items = True
    # The doc is already a dictionary, no need for to_dict()
    print(f"Name: {doc['Name']}")
    print(f"Category: {doc['Category']}")
    print(f"Price: ${doc['Price']:.2f}")
    print(f"Description: {doc['Description']}")
    print("-" * 50)

if not found_items:
    print("No matching items found.")

## 2. Create Agent With AI Search Tool
We'll create a new agent and attach an `AzureAISearchTool` referencing **myfitnessindex**.
In your environment, you need:
- `PROJECT_CONNECTION_STRING` - from your AI Foundry project overview
- `MODEL_DEPLOYMENT_NAME` - from the deployed model name

Let's initialize the `AIProjectClient` with `DefaultAzureCredential`.


In [ ]:
# Import required libraries:
# - os: For accessing environment variables
# - DefaultAzureCredential: Azure's authentication mechanism
# - AIProjectClient: Main client for interacting with AI Projects
# - AzureAISearchTool & ConnectionType: Used to configure search capabilities
import os
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureAISearchTool, ConnectionType

# Initialize the AI Project Client which we'll use to:
# 1. Connect to our Azure AI project
# 2. Create agents with search capabilities
# 3. Manage project resources
try:
    project_client = AIProjectClient.from_connection_string(
        # Use Azure's default authentication method
        credential=DefaultAzureCredential(),
        # Connect using the project connection string from environment variables
        conn_str=os.environ["PROJECT_CONNECTION_STRING"],
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"❌ Error initializing project client: {e}")

### Find (or create) the Azure AI Search connection in your Foundry project
We'll now use `project_client.connections.get_default(...)` to retrieve the default Azure AI Search connection, **including** credentials.


In [ ]:
# Try to get the default Azure AI Search connection from our project
# - Azure AI Search (formerly Cognitive Search) is a cloud search service that helps 
#   us add search capabilities to our applications
# - The connection contains endpoint and credential information needed to access the search service
search_conn = project_client.connections.get_default(
    # Specify we want an Azure AI Search connection type
    connection_type=ConnectionType.AZURE_AI_SEARCH,
    # include_credentials=True means we'll get the full connection info including auth keys
    include_credentials=True
)

# Check if we found a connection
if not search_conn:
    print("❌ No default Azure AI Search connection found in your project.")
else:
    # If found, print the connection details
    # - The connection ID is a unique identifier for this connection in our project
    # - The endpoint URL is where our search service is hosted
    print(f"Found default Azure AI Search connection ID: {search_conn.id}")
    print(f"Index endpoint: {search_conn.endpoint_url}")

### Create the Agent with `AzureAISearchTool`
We'll attach the tool, specifying the index name we created.


In [ ]:
# Get the model deployment name from environment variables
# This is the Azure OpenAI model we'll use for our agent
model_name = os.environ.get("MODEL_DEPLOYMENT_NAME")
agent = None

if search_conn:
    # Create an Azure AI Search tool that will allow our agent to search the fitness equipment index
    # - The tool needs the connection ID we got earlier to authenticate
    # - index_name specifies which search index to query (we created myfitnessindex earlier)
    ai_search_tool = AzureAISearchTool(
        index_connection_id=search_conn.id,
        index_name=index_name
    )

    # Create an AI agent that can understand natural language and search our index
    # - The agent uses our Azure OpenAI model for natural language understanding
    # - We give it instructions to act as a fitness shopping assistant
    # - We attach the search tool so it can look up products
    # - tool_resources provides the connection details the tool needs
    agent = project_client.agents.create_agent(
        model=model_name,
        name="fitness-agent-search",
        instructions="""
        You are a Fitness Shopping Assistant. You help users find items, but always disclaim not to provide medical advice.
        """,
        tools=ai_search_tool.definitions,
        tool_resources=ai_search_tool.resources,
        headers={"x-ms-enable-preview": "true"},  # Enable preview features
    )
    print(f"🎉 Created agent, ID: {agent.id}")

## 3. Run a Conversation with the Agent
We'll open a new thread, post a question, and let the agent search the index for relevant items.

In [ ]:
def run_agent_query(question: str):
    # Step 1: Create a new conversation thread
    # In Azure AI Agent service, conversations happen in threads, similar to chat conversations
    # Each thread can contain multiple back-and-forth messages
    thread = project_client.agents.create_thread()
    print(f"📝 Created thread, ID: {thread.id}")

    # Step 2: Add the user's question as a message in the thread
    # Messages have roles ("user" or "assistant") and content (the actual text)
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=question
    )
    print(f"💬 Created user message, ID: {message.id}")

    # Step 3: Create and start an agent run
    # This tells the agent to:
    # - Read the user's message
    # - Use its AI Search tool to find relevant products
    # - Generate a helpful response
    run = project_client.agents.create_and_process_run(
        thread_id=thread.id,
        assistant_id=agent.id
    )
    print(f"🤖 Agent run status: {run.status}")

    # Check for any errors during the agent's processing
    if run.last_error:
        print("⚠️ Run error:", run.last_error)

    # Step 4: Get the agent's response
    # Retrieve all messages and find the most recent assistant response
    # The response might contain multiple content blocks (text, images, etc.)
    msg_list = project_client.agents.list_messages(thread_id=thread.id)
    for m in reversed(msg_list.data):
        if m.role == "assistant" and m.content:
            print("\nAssistant says:")
            for c in m.content:
                if hasattr(c, "text"):
                    print(c.text.value)
            break

# Try out our agent with two example queries:
# 1. A general question about strength training equipment
# 2. A specific request for cardio equipment with a price constraint
if agent:
    run_agent_query("Which items are good for strength training?")
    run_agent_query("I need something for cardio under $300, any suggestions?")

## 4. Cleanup
We'll clean up the agent. (In production, you might want to keep it!)

In [ ]:
if agent:
    project_client.agents.delete_agent(agent.id)
    print("🗑️ Deleted agent")

index_client.delete_index(index_name)
print(f"🗑️ Deleted index {index_name}")

# 🎉 Congrats!
You've successfully:
1. **Created** an Azure AI Search index programmatically.
2. **Populated** it with sample fitness data.
3. **Created** an Agent that queries the index using `AzureAISearchTool`.
4. **Asked** the agent for item recommendations.

Continue exploring how to integrate **OpenTelemetry** or the `azure-ai-evaluation` library for advanced tracing and evaluation capabilities. Have fun, and stay fit! 🏆